In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

#Read data from chunks csv file
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/chunks.csv')

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 22.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Function to get spaCy embedding for a text string
def get_embedding(text):
    doc = nlp(text)
    # Use the vector for the entire document
    return doc.vector

# Apply the function to the 'text' column and create a new 'embedding' column
df['embedding'] = df['text'].apply(get_embedding)


In [ ]:
# Display the DataFrame with embeddings
df

,text,section,article,number1,number2,number3,number4,number5,number6,CELEX number,embedding
0,2023/2534,Document,None,None,None,None,None,None,None,32023R2534,"[-1.112007, 0.23834589, -1.8763049, 0.96194446..."
1,22.11.2023,Document,None,None,None,None,None,None,None,32023R2534,"[-1.0941092, -0.48539665, -0.982906, -0.159265..."
2,COMMISSION DELEGATED REGULATION (EU) 2023/2534,Document,None,None,None,None,None,None,None,32023R2534,"[-0.47455367, -0.49320582, -0.47712865, 0.1735..."
3,of 13 July 2023,Document,None,None,None,None,None,None,None,32023R2534,"[-0.2924167, -0.4822841, -0.90270233, 0.232256..."
4,supplementing Regulation (EU) 2017/1369 of the...,Document,None,None,None,None,None,None,None,32023R2534,"[-0.05712086, -0.20380268, -0.23981985, -0.049..."
...,...,...,...,...,...,...,...,...,...,...,...
28201,This Decision shall enter into force on the tw...,Document,4,None,None,None,None,None,None,32017D0089,"[0.12472073, -0.35735857, -0.07501161, 0.07295..."
28202,"Done at Brussels, 17 January 2017.",Document,4,None,None,None,None,None,None,32017D0089,"[-0.36624625, -0.42442584, -0.15929836, -0.095..."
28203,For the Commission,Document,4,None,None,None,None,None,None,32017D0089,"[-0.016838193, -0.88174415, -0.2812731, 0.5818..."
28204,The President,Document,4,None,None,None,None,None,None,32017D0089,"[-0.44046122, -0.28894782, -1.1724594, 1.35895..."


In [ ]:
#storing data in df with embeddings in csv file 
df.to_csv('/content/drive/MyDrive/INLPT_Project/csv_files/df.csv')

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import faiss

# Find the maximum length of embeddings
max_length = df['embedding'].apply(len).max()
print(max_length)

# Pad or truncate embeddings to a fixed size
df['embedding'] = df['embedding'].apply(lambda emb: np.pad(emb, (0, max_length - len(emb)), 'constant'))

# Convert the list of embeddings to a 2D numpy array
embeddings_array = np.vstack(df['embedding'].to_numpy())

# Check if there are valid embeddings
if embeddings_array.shape[0] == 0:
    print("No valid embeddings found.")
else:
    # Print the embeddings
    print(embeddings_array)
    print(embeddings_array.shape)

    # Initialize Faiss index
    index = faiss.IndexFlatL2(embeddings_array.shape[1])

    # Add embeddings to the index
    index.add(embeddings_array)



# Save Faiss index and embeddings_array for future use
faiss.write_index(index, '/content/drive/MyDrive/faiss_index.index')
np.save('/content/drive/MyDrive/faiss_embeddings.npy', embeddings_array)


96
[[-1.112007    0.23834589 -1.8763049  ...  0.10647964 -0.08213249
   0.569291  ]
 [-1.0941092  -0.48539665 -0.982906   ...  0.268071   -0.61646414
   1.8584887 ]
 [-0.47455367 -0.49320582 -0.47712865 ... -0.32960764  0.51401174
   0.7988134 ]
 ...
 [-0.01683819 -0.88174415 -0.2812731  ... -0.38634834  0.3355708
  -0.10941356]
 [-0.44046122 -0.28894782 -1.1724594  ... -0.26230073  0.39571342
   0.67738014]
 [-0.6500286  -1.2508626   1.0812093  ... -0.04203786  0.02462721
   1.0423484 ]]
(28206, 96)


In [39]:
# Provide a query text
query_text = "What is the Maximum Total Harmonic Distortion for the market of North America and Taiwan?"

# Get the embedding for the query text
query_embedding = get_embedding(query_text)


In [41]:
import numpy as np
import faiss
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# Load Spacy model
nlp = spacy.load("en_core_web_md")

# Load Faiss index
index = faiss.read_index('/content/drive/MyDrive/faiss_index.index')

# Load embeddings array
embeddings_array = np.load('/content/drive/MyDrive/faiss_embeddings.npy')

"""
def similarity_search(query_embedding, index, embeddings_array, k=5):
    # Find the k most similar embeddings to the query using Faiss
    _, indices = index.search(np.array([query_embedding]), k)

    # Extract the corresponding text from the DataFrame
    similar_texts = df.iloc[indices[0]]['text'].tolist()

    return similar_texts
"""

# Calculate cosine similarity between the query and all embeddings in the array
similarity_scores = cosine_similarity([query_embedding], embeddings_array)[0]

# Rank array based on similarity scores
ranked_indices = np.argsort(similarity_scores)[::-1]


In [42]:
# Print the results
print("Top 5 Similar Embeddings:")
for i, index in enumerate(ranked_indices[:5], start=1):
    print(f"{i}. Similarity Score: {similarity_scores[index]}")
    print(embeddings_array[index])
    print()

Top 5 Similar Embeddings:
1. Similarity Score: 0.8739408254623413
[ 0.25528887 -0.32756254 -0.13168132  0.48058492  0.11488687  0.31964552
 -0.04186747  0.08298364  0.16279967  0.45854387  0.10629068  0.1752586
 -0.69076926 -0.5512621  -0.38228962 -0.386802   -0.10884034  0.3515894
 -0.17419265 -0.31644443 -0.64504284 -0.10663877  0.16494018  0.03655768
  0.2972049   0.16807494  0.9580058   0.1899414   0.22810198  0.29022
 -0.19624037 -0.6576434   0.00889968  0.25805736 -0.17816843 -0.28022742
  0.63180023  0.16903344 -0.3377043  -0.26695538 -0.20208855 -0.02199562
 -0.297605    0.18680783 -0.1841241   0.25557208  0.23272711 -0.08805089
 -0.17689614  0.16880195 -0.6497377   0.29288742  0.47946286 -0.13799162
 -0.5896655   0.15445419 -0.04652969 -0.02055495  0.40888023 -0.07976172
 -0.12779415 -0.6926056   0.05361414 -0.13835546 -0.42229918 -0.31376594
  0.44481266  0.10690737  0.13421072 -0.16828905  0.21741518  0.29777858
  0.76867425 -0.19014552  0.17127499 -0.41373622 -0.5828734  -0

In [43]:
# Print the text for the top 5 similar embeddings
print("Top 5 Similar Texts:")
for i, index in enumerate(ranked_indices[:5], start=1):
    print(f"{i}. Similarity Score: {similarity_scores[index]}")
    print(f"   Text: {df['text'].iloc[index]}")
    print()

Top 5 Similar Texts:
1. Similarity Score: 0.8739408254623413
   Text: The Memorandum of Understanding in annex between the European Commission and the International Atomic Energy Agency concerning the EURDEP (EUropean Radiological Data Exchange Platform) is approved.

2. Similarity Score: 0.8710848093032837
   Text: The signing of the Framework Agreement and its Protocol on a Multilateral Nuclear Environmental Programme in the Russian Federation and its Protocol on Claims, Legal Proceedings and Indemnification is hereby approved on behalf of the European Community, subject to the Council Decision concerning the conclusion of the said Agreement.

3. Similarity Score: 0.8696929812431335
   Text: between the European Atomic Energy Community and the Cabinet of Ministers of Ukraine for Co-operation in the Peaceful Uses of Nuclear Energy

4. Similarity Score: 0.8683764934539795
   Text: HAVING REGARD to the Agreement for Cooperation between the Government of Japan and the European Atomic Ene